# Task 

This notebooks merges HRS and SHARE for estimation of moments. 

In [ ]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt

In [15]:
hrs = pd.read_pickle('../data_sources/hrs/rndhrs_wide_select.pkl')
sh = pd.read_pickle('../data_sources/share/share_wide_select.pkl')
df = hrs.append(sh)
df.groupby('country').count()['rage_w1']

country
10    17823
11     1893
12     3007
13     3053
14     2972
15     2396
16     2559
17     3193
18     1707
19     2898
20     1003
23     3827
25     2595
Name: rage_w1, dtype: int64

In [16]:
hrs = pd.read_pickle('../data_sources/hrs/rndhrs_wide_select.pkl')
sh = pd.read_pickle('../data_sources/share/share_wide_select.pkl')
df = hrs.append(sh)
df.groupby('country').count()['rage_w1']
cnums = [10,12,13,14,15,16,17,18]
df = df[df.country.isin(cnums)]
df.groupby('country').count()['rage_w1']

country
10    17823
12     3007
13     3053
14     2972
15     2396
16     2559
17     3193
18     1707
Name: rage_w1, dtype: int64

We then select the countries used in the paper. 


In [17]:
cnums = [10,12,13,14,15,16,17,18]
df = df[df.country.isin(cnums)]
df.groupby('country').count()['rage_w1']

country
10    17823
12     3007
13     3053
14     2972
15     2396
16     2559
17     3193
18     1707
Name: rage_w1, dtype: int64

We select the age range. 

In [18]:
df = df[(df['rage_w1']>=50) & (df['rage_w1']<=75)]


We create the mapping to country numbers from 1 to 8 that we use in paper

In [19]:
df['cid'] = df['country'].replace({12:1,13:2,14:3,15:4,16:5,17:6,18:7,10:8})
cnames = ['DE','SE','NL','SP','IT','FR','DK','US']
cids = [x for x in range(1,9)]
map_cnames = dict(zip(cids,cnames))
df['cname'] = df['cid'].replace(map_cnames)
df.groupby('cname').count()['rage_w1']
df['share'] = np.where(df['cid']!=8,1,0)

We compute the transition variables, whether in good health in wave 2 dependent on whether the respondent is in poor or good health in wave 1. 

In [20]:
map_adla = {0:0,1:1,2:1,3:1,4:1,5:1}
df['radla_w1'] = df['radla_w1'].replace(map_adla)
df['radla_w2'] = df['radla_w2'].replace(map_adla)

In [21]:
df['gg2'] = np.where(df['radla_w1']==0,1-df.loc[:,'radla_w2'],np.nan)
df['gb2'] = np.where(df['radla_w1']==1,1-df.loc[:,'radla_w2'],np.nan)
df['g_w1'] = 1-df.loc[:,'radla_w1']
df['g_w2'] = 1-df.loc[:,'radla_w2']

In [22]:
df.to_pickle('../data_sources/hrs-share_wide_select.pkl')